In [12]:
from shapely import geometry, wkt
import rasterio
import numpy as np
import rasterio.features
import geopandas as gpd
import fiona  
from rasterio import features as rio_features 
from shapely.geometry import Polygon, box
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas import GeoSeries, GeoDataFrame

# multiple classes
vector_file = 'veg_class.geojson' 
raster_file = 'WorldCover.tif' 
rasterOuput = 'rasterized_crop.tif'
out_dir = ''


# 1) Rasterize Vector file - Values saved in attribute column

In [7]:

# Read the vector file as a GeoDataFrame
gdf = gpd.read_file(vector_file)

with rasterio.open(raster_file) as raster:
    raster_shape      = raster.shape
    raster_transform  =  raster.transform
    raster_profile    = raster.profile
    

#with fiona.open(vectorfile, "r") as vector:
geom_value = ((geom,value) for geom, value in zip(gdf.geometry, gdf.Class))
raster = rio_features.rasterize(geom_value,
                                out_shape=raster_shape,
                                transform=raster_transform,
                                all_touched=True,
                                fill=0,
                                dtype=np.uint8)   
    
raster_profile.update(nodata=0, dtype= 'uint8')
   
out_filename = rasterOuput

print(raster_profile)
print(raster.min(), raster.max())

with rasterio.open(out_filename, "w", **raster_profile) as dst:
        dst.write(raster, 1) 

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0, 'width': 72000, 'height': 36000, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(8.333333333333333e-05, 0.0, 6.0,
       0.0, -8.333333333333333e-05, 51.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
0 103


# 2) Burn the rasterized classes to a tif file (World Cover)

In [ ]:
import rasterio
from rasterio.merge import merge

# Open the two raster files to be merged
raster1 = rasterio.open('WorldCover.tif')
raster2 = rasterio.open('rasterized_crop.tif')

# Read the metadata of the first raster
meta = raster1.meta.copy()
print(meta)

# Create a list of the two raster datasets
datasets = [raster2, raster1]

# Perform the merge
mosaic, _ = merge(datasets)

mosaic_shape= mosaic.shape
print(mosaic_shape)

# Update the metadata with the merged shape and data type
meta.update(count=1)

# Update the data type of the merged mosaic to match the first raster
mosaic = mosaic.astype(raster1.meta["dtype"])

# Save the merged raster
merged_path = 'WorldCover_reclass.tif'
with rasterio.open(merged_path, 'w', **meta) as dst:
    dst.write(mosaic,)


{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 72000, 'height': 36000, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(8.333333333333333e-05, 0.0, 6.0,
       0.0, -8.333333333333333e-05, 51.0)}


# 3) Compressed a tif file

In [ ]:

dat = rasterio.open("WorldCover_reclass.tif")
profile = dat.profile.copy()
profile.update(compress='lzw')

with rasterio.open("WorldCover_compressed.tif", 'w', **profile) as dst:
    for ji, window in dat.block_windows(1):
        dst.write(dat.read(window=window), window=window)